## Sequntial API

사용할 패키지를 불러옵니다.

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from utils import standard_scaling, R_squared, train_val_split
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

캘리포니아 집값 데이터 클래스를 불러옵니다.

In [2]:
housing = fetch_california_housing()

데이터를 훈련, 검증, 테스트 데이터로 나눕니다.

In [3]:
x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full)

입력값들을 정규화 시켜줍니다.

In [9]:
x_train = standard_scaling(x_train)
x_val = standard_scaling(x_val)
x_test = standard_scaling(x_test)

y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)
y_test = y_test.reshape(-1,1)

Sequential api로 keras모델을 만들되 하이퍼파라미터를 조절가능한 형태의 모델로 만들기 위해 모델을 만드는 함수를 만듭니다.

In [24]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    """
    args:
        n_hidden: 은닉층의 개수 (하이퍼파라미터)
        n_neurons: 각 은닉층의 유닛 갯수 (하이퍼파라미터)
        learning_rate: 최적화 기법의 학습률 (하이퍼파라미터)
        input_shape: 모델에 들어가는 입력변수의 개수
    return:
        keras model
    """
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

kears model을 회귀모델로 사용하겠습니다.

In [25]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
# # 학습 시작
# keras_reg.fit(x_train, 
#               y_train,
#               epochs=100,
#               validation_data=(x_val, y_val),
#               callbacks=[keras.callbacks.EarlyStopping(patience=10)]
#               )
# # 점수 출력
# mse_test = keras_reg.score(x_test, y_test)
# y_pred = keras_reg.predict(x_test)

하이퍼파라미터 조정을 위한 sklearn의 RandomizedSearchCV 적용하겠습니다.

In [28]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden":(0,1,2,3), # 은닉층 개수
    "n_neurons": np.arange(20,50).tolist(), # 은닉층의 유닛 개수
    "learning_rate": np.arange(0.01,0.02).tolist() # 학습률
}
# 랜덤서치cv 기법을 10의 에폭과 cross validation은 3등분 합니다.
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=3, cv=2)
# 각 하이퍼파라미터를 적용한 모델 학습을 시작합니다.
rnd_search_cv.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    callbacks=[keras.callbacks.EarlyStopping(patience=3)]
)

363/363 [==============================] - 1s 2ms/step - loss: 0.7675 - val_loss: 0.4377


RandomizedSearchCV(cv=2,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x00000157C70A4FD0>,
                   n_iter=3,
                   param_distributions={'learning_rate': [0.01],
                                        'n_hidden': (0, 1, 2, 3),
                                        'n_neurons': [20, 21, 22, 23, 24, 25,
                                                      26, 27, 28, 29, 30, 31,
                                                      32, 33, 34, 35, 36, 37,
                                                      38, 39, 40, 41, 42, 43,
                                                      44, 45, 46, 47, 48, 49]})

최적의 하이퍼파라미터를 출력합니다.

In [29]:
rnd_search_cv.best_params_ 

{'n_neurons': 38, 'n_hidden': 3, 'learning_rate': 0.01}

다양한 조건의 모델중에서 가장 성능이 좋은 모델의 점수를 출력합니다.

In [30]:
rnd_search_cv.best_score_

-0.5963866412639618

최적의 파라미터를 가진 모델을 저장합니다.

In [33]:
best_model = rnd_search_cv.best_estimator_.model

### 하이퍼파라미터
- 은닉층의 개수
- 은닉층의 유닛 개수
- 학습률
- 배치 크기
- 옵티마이저
- 활성화 함수
- 반복 횟수

#### 은닉층의 개수

- 은닉층의 갯수가 많아질수록 깊은 네트워크를 가진 심층 신경망이라고 부릅니다. 복잡한 문제를 해결하는데는 얕은 신경망 보다는 심층 신경망이 파라미터 효율성이 좋고 동일한 양의 훈련 데이터에서 더 높은 성능을 부릅니다.  
- 은닉층은 입력층과 가까울수록 단순한 패턴을 모델링하고 출력층과 가까울수록 복잡한 패턴을 모델링합니다.
- 단순한 패턴이 잘 학습된 모델을 일부 복잡한 패턴을 따로 학습시키는 전이학습 방법을 사용하여 다양한 문제를 해결하기 위한 기본 토대가 되어질 수도 있습니다.

#### 은닉층의 유닛개수

- 은닉층의 일반적인 유닛개수 구성방식은 각층의 뉴런을 점점 줄여서 깔때기 처럼 구성합니다. 이로인하여 저수준의 많은 특성이 고수준의 적은 특성으로 합쳐질 수 있기 떄문입니다.
- 하지만 최근에는 모든 층의 유닛개수를 동일하게 하거나 점점 더 많아지는 방식으로도 성능개선을 보입니다.
- 처음 첫번째 은닉층의 유닛개수는 여유있게 두는 것이 좋습니다.
- 실전에서는 많은 수의 은닉층과 유닛개수를 사용하고 조기종료나 규제기법을 사용하여 찾아가는 방식의 스트레치 팬츠(stretch pants)방식이 자주 사용되어 집니다.

#### 학습률

- 매우 낮은 학습률에서 시작하여 점진적으로 매우 큰 학습률까지 반복적으로 모델을 훈련하는 방법이 있습니다.
- 최적의 학습률은 손실이 다시 상승하는 지점보다 조금 아래에 있습니다.

#### 배치 크기

- 큰 배치 크기를 사용하면 GPU와 같은 하드웨어 가속기를 효율적으로 활용이 가능합니다.
- 배치 크기가 커지면 한번에 처리할 데이터 수가 증가하므로 초당 더 많은 데이터를 처리할 수 있습니다.
- Yan Lecun 2~32개의 배치크기가 좋다
- 학습률 예열을 사용하여 큰 배치 크기를 시도하기도 합니다.

#### ETC

Leslie Smith의 2018년 논문에 신경망 하이퍼파라미터 튜닝에 관한 좋은 모범 사례가 소개되어 있음